In [44]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from rouge import Rouge
import nltk
from nltk.tokenize import word_tokenize

In [45]:
# считываем таблицу в df
df = pd.read_excel(r'C:\Users\nev1L\Downloads\Test_check.xlsx',"sasha_3") # считываем таблицу в df
df.head(10)

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп..."
5,https://lenta.ru/articles/2025/03/09/aslan/,"20 лет назад, 8 марта 2005 года, в чеченском с..."
6,https://expert.ru/finance/investorov-svyazyvay...,"Говоря сухим языком определений, цифровые фина..."
7,https://ria.ru/20250123/svo-1985822676.html,Поддержка военнослужащих и их семей с самого н...
8,https://expert.ru/mnenie/vse-teper-zavisit-tol...,— За последние два с половиной года из-за введ...
9,https://ria.ru/docs/about/privacy_policy.html,Федеральное государственное унитарное предприя...


In [46]:
def clean_text(text):

    # удаляем ненужные символы
    text = text.replace('\xa0', ' ')
    text = text.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

In [47]:
def parse_news_text(s): 

    # получаем html-страницу    
    req = requests.get(s)
    html = req.text

    #считываем нужный нам текст
    soup = BeautifulSoup(html, 'lxml')
    
    if 'ria.ru' in s:
        rows = soup.find_all('div', {'itemprop': 'articleBody'})
    else:
        rows = soup.find_all('p')

    text = ''
    ind = len(rows) - 1

    if 'expert.ru' in s:
        for i in range(len(rows)):
            if i != ind:
                text = text + " " + rows[i].get_text()
    else:
        for i in range(len(rows)):
            text = text + " " + rows[i].get_text()
            
    return clean_text(text)

In [48]:
# добавляем столбец который содержит спарсенный текст
df['sparsely_text'] = df['URL'].apply(parse_news_text)

In [49]:
def calculate_precision(reference, hypothesis):
    
    # Токенизация (разбиение на слова)
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())

    # Множества слов для сравнения
    ref_set = set(ref_tokens)
    hyp_set = set(hyp_tokens)

    # True Positives (правильно извлеченные слова)
    tp = len(ref_set & hyp_set)

    precision = tp / len(hyp_set) if hyp_set else 0

    return precision

In [50]:
def calculate_recall(reference, hypothesis):
    # Токенизация (разбиение на слова)
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())

    # Множества слов для сравнения
    ref_set = set(ref_tokens)
    hyp_set = set(hyp_tokens)

    # True Positives (правильно извлеченные слова)
    tp = len(ref_set & hyp_set)

    recall = tp / len(ref_set) if ref_set else 0

    return recall

In [51]:
def calculate_f1(precision, recall):

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1

In [52]:
# скачиваем модель для токенизации
nltk.download('punkt') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nev1L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
# вычисляем precision для каждой новости 
df["precision"] = df.apply(lambda row: calculate_precision(row["lib_text"], row["sparsely_text"]), axis=1)
# вычисляем recall для каждой новости
df["recall"] = df.apply(lambda row: calculate_recall(row["lib_text"], row["sparsely_text"]), axis=1)
# вычисляем f1 для каждой новости
df["f1"] = df.apply(lambda row: calculate_f1(row["precision"], row["recall"]), axis=1)

In [54]:
def find_differnce(reference, hypothesis):

    # токенизация
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())

    # поиск отсутствующих слов
    missing_words = [word for word in ref_tokens if word not in hyp_tokens]
    
    return missing_words

In [55]:
def find_len_differnce(reference, hypothesis):

    # токенизация
    ref_tokens = word_tokenize(reference.lower())
    hyp_tokens = word_tokenize(hypothesis.lower())

    # поиск отсутствующих слов
    missing_words = [word for word in ref_tokens if word not in hyp_tokens]
    
    return len(missing_words)

In [56]:
# добавляем столбец со словами, которые не получилось спарсить
df["missing_words"] = df.apply(lambda row: find_differnce(row["lib_text"], row["sparsely_text"]), axis=1)
# добавляем стоблбец с колличеством слов, , которые не получилось спарсить
df["missing_len"] = df.apply(lambda row: find_len_differnce(row["lib_text"], row["sparsely_text"]), axis=1)

In [57]:
df

,URL,lib_text,sparsely_text,precision,recall,f1,missing_words,missing_len
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,Серьезнее других от введения западных санкций ...,0.980459,0.999134,0.989708,[общий],1
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","МОСКВА, 25 апр — РИА Новости. Ключевая процент...",0.944444,0.931731,0.938045,"[фев, депозитах, центробанка., fontvielle, пок...",82
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,Со следующего года начнется реализация 12 мега...,0.988930,0.980249,0.984570,"[обеспечивает, российским, инвестирует, бюджет...",28
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"МОСКВА, 4 мар - РИА Новости. Самозапрет на кре...",0.904255,0.967008,0.934579,"[законодательная, населения, права, контроля, ...",30
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","Заходя в магазин, вы часто замечали зеленые уп...",0.973435,1.000000,0.986538,[],0
5,https://lenta.ru/articles/2025/03/09/aslan/,"20 лет назад, 8 марта 2005 года, в чеченском с...",Фото: Машатин Владимир / PhotoXPress 20 лет на...,0.848146,0.992629,0.914717,"[*, *, *, полковник, госизменой, власть, радик...",11
6,https://expert.ru/finance/investorov-svyazyvay...,"Говоря сухим языком определений, цифровые фина...","Говоря сухим языком определений, цифровые фина...",0.969887,0.978481,0.974165,"[токены, становились, аналогом, структурной, о...",17
7,https://ria.ru/20250123/svo-1985822676.html,Поддержка военнослужащих и их семей с самого н...,"МОСКВА, 23 янв - РИА Новости. Выплаты участник...",0.916841,0.910479,0.913649,"[поддержка, сейчас, довольствием, среднее, инв...",153
8,https://expert.ru/mnenie/vse-teper-zavisit-tol...,— За последние два с половиной года из-за введ...,— За последние два с половиной года из-за введ...,0.968835,0.961022,0.964912,"[чистые, материалы, сотни, наименований, этом,...",30
9,https://ria.ru/docs/about/privacy_policy.html,Федеральное государственное унитарное предприя...,Федеральное государственное унитарное предприя...,0.805022,0.858268,0.830793,"[стандартам, понимания, языки, процессы, обрат...",223


In [59]:
df.to_excel('output.xlsx', index=False)

Предложения по улучшению:

1. В некоторых статьях отсутсвуют целые абзацы, поэтому следует добаавить еще теги для парсинга

2. Нужна более глубокая настройка парсера, например использование какой нибудь модели, которая будет определять является ли информация в теге нужной или нет

3. также вроде есть ошибка в исзодных данных, т.к. в источнике РИА новости статьи начинаются с даты ее публикации. Но в каких-то эталонных текстах дата присутствует, в каких-то ее нет